In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:100%;}
div.output_subarea output_text output_stream output_stdout{width:97%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:14pt;}
div.text_cell_render.rendered_html{font-size:14pt;}
div.text_cell_render ul li, code{font-size:22pt; line-height:14px;}
div.output {font-size:14pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:14pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:14pt;padding:5px;}
table.dataframe{font-size:14px;}
</style>
"""))

## RAG 절차

1. 문서를 읽는다
    %pip install --upgrade --quiet docx2txt
2. 문서를 쪼갠다
    %pip install -qU langchain-text-splitters (오버랩되게 알아서 자른다/문장단위로 자르거나)
3. 쪼갠 문서를 임베딩하여 vector database에 넣음(local에 저장) cf. 클라우드에 저장
    %pip install -q langchain-chroma
4. 질문을 이용해 유사도 검색
5. 유사도 검색한 문서를 LLM에 질문과 함께 전달하여 답변 얻음(랭체인 사용 가능)
    %pip install -q langchain   
    (https://smith.langchain.com 에서 key생성  .env에 LANGCHAIN_API_KEY로 추가)(실무에서 TRACE)    

# 0. 패키지 설치

In [ ]:
# 문서읽어오기
%pip install --upgrade --quiet docx2txt

In [2]:
# 텍스트를 chunk로 나누는 기능만 있는 경량 모듈
%pip install -qU langchain-text-splitters 

Note: you may need to restart the kernel to use updated packages.


In [4]:
# 벡터DB(로컬DB) 어제의 chromadb가 아님
%pip install -q langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [5]:
# langchain 사용
%pip install -q langchain

Note: you may need to restart the kernel to use updated packages.


# 1. 문서읽기(X)

In [7]:
%%time
from langchain_community.document_loaders import Docx2txtLoader
loader = Docx2txtLoader("./data/소득세법(법률)(제21065호)(20260102).docx")
document = loader.load()

CPU times: total: 4.3 s
Wall time: 4.65 s


In [9]:
len(document)

1

# 2. 문서를 쪼개면서 읽기(O)
- https://docs.langchain.com/oss/python/integrations/splitters

## 2.1 1500토큰씩 쪼개서 읽어오기

In [10]:
import time
start = time.time()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import TokenTextSplitter
loader = Docx2txtLoader("./data/소득세법(법률)(제21065호)(20260102).docx")
# document = loader.load() - 한덩어리로 읽어오니까 X
# gpt-4, gpt-4o, gpt-4 tubo, gpt4o-mini, embedding모델들은 다 같은 방식으로 토큰 추출
text_splitter = TokenTextSplitter(
    encoding_name="cl100k_base", #토큰을 세는 방식 이름
    chunk_size=1500,             #chunk당 토큰 수  기준
    chunk_overlap=200
    # separators = ["\n", "\n\n"]  파라미터가 없음
)
documents = loader.load_and_split(text_splitter=text_splitter)
runtime = time.time() - start
print("문서를 쪼개면서 읽는 시간: ", runtime)

문서를 쪼개면서 읽는 시간:  5.915306091308594


In [12]:
len(documents)  # chunk갯수

180

In [16]:
# chunk 글자수
# documents[0].page_content
print([len(document.page_content) for document in documents])

[1699, 1656, 1641, 1650, 1738, 1442, 1287, 1535, 1325, 1619, 1596, 1588, 1566, 1639, 1622, 1559, 1612, 1638, 1573, 1465, 1436, 1609, 1456, 1497, 1635, 1606, 1533, 1649, 1662, 1595, 1603, 1678, 1595, 1637, 1601, 1539, 1561, 1594, 1693, 1708, 1657, 1627, 1636, 1659, 1667, 1595, 1491, 1485, 1645, 1709, 1629, 1617, 1495, 1626, 1612, 1620, 1609, 1576, 1636, 1602, 1556, 1563, 1600, 1616, 1643, 1691, 1635, 1685, 1621, 1631, 1609, 1605, 1603, 1604, 1698, 1686, 1702, 1612, 1539, 1558, 1651, 2060, 1562, 1606, 1557, 1648, 1594, 1615, 1766, 1651, 1690, 1576, 1536, 1553, 1638, 1685, 1693, 1694, 1664, 1529, 1627, 1703, 1675, 1546, 1585, 1687, 1679, 1714, 1603, 1655, 1648, 1495, 1531, 1562, 1594, 1646, 1543, 1449, 1593, 1559, 1521, 1473, 1519, 1545, 1668, 1700, 1692, 1655, 1648, 1741, 1670, 1628, 1639, 1623, 1638, 1642, 1666, 1658, 1594, 1591, 1561, 1641, 1498, 1610, 1567, 1613, 1636, 1619, 1531, 1496, 1702, 1598, 1579, 1627, 1559, 1585, 1665, 1565, 1616, 1564, 1612, 1535, 1512, 1557, 1576, 1628, 165

In [19]:
# chunk 글자수 최대값, 최소값
print(max([len(document.page_content) for document in documents]))
print(min([len(document.page_content) for document in documents]))

2060
955


In [20]:
print(documents[0].page_content)

소득세법

소득세법

[시행 2026. 1. 2.] [법률 제21065호, 2025. 10. 1., 타법개정]

기획재정부(재산세제과(양도소득세)) 044-215-4312

기획재정부(소득세제과(근로소득)) 044-215-4216

기획재정부(금융세제과(이자소득, 배당소득)) 044-215-4233

기획재정부(소득세제과(사업소득, 기타소득)) 044-215-4217



제1장 총칙 <개정 2009. 12. 31.>



제1조(목적) 이 법은 개인의 소득에 대하여 소득의 성격과 납세자의 부담능력 등에 따라 적정하게 과세함으로써 조세부담의 형평을 도모하고 재정수입의 원활한 조달에 이바지함을 목적으로 한다.

[본조신설 2009. 12. 31.]

[종전 제1조는 제2조로 이동 <2009. 12. 31.>]



제1조의2(정의) ① 이 법에서 사용하는 용어의 뜻은 다음과 같다. <개정 2010. 12. 27., 2014. 12. 23., 2018. 12. 31.>

1. “거주자”란 국내에 주소를 두거나 183일 이상의 거소(居所)를 둔 개인을 말한다.

2. “비거주자”란 거주자가 아닌 개인을 말한다.

3. “내국법인”이란 「법인세법」 제2조제1호에 따른 내국법인을 말한다.

4. “외국법인”이란 「법인세법」 제2조제3호에 따른 외국법인을 말한다.

5. “사업자”란 사업소득이 있는 거주자를 말한다.

② 제1항에 따른 주소ㆍ거소와 거주자ㆍ비거주자의 구분은 대통령령으로 정한다.

[본조신설 2009. 12. 31.]



제2조(납세의무) ① 다음 각 호의 어느 하나에 해당하는 개인은 이 법에 따라 각자의 소득에 대한 소득세를 납부할 의무를 진다.

1. 거주자

2. 비거주자로서 국내원천소득(國內源泉所得)이 있는 개인

② 다음 각 호의 어느 하나에 해당하는 자는 이 법에 따라 원천징수한 소득세를 납부할 의무를 진다.

1. 거주자

2. 비거주자

3. 내국법인

4. 외국법인의 국내지점 또는 국내영업소(출장소, 그 밖에 이에 준하는 것을 포함한다. 

## 2.2 1500글자씩 쪼개서 읽어오기

In [2]:
import time
start = time.time()
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter  # 글자수로 쪼개는
loader = Docx2txtLoader("data/소득세법(법률)(제21065호)(20260102).docx")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,   # 문서를 쪼갤 때 1500글자씩 chunking
    chunk_overlap=200, # 0~1500
    # separators=["\n\n", "\n", " ", ""]  지정하지 않으면 디폴트값
)
# 재귀적으로 다음 순서대로 시도:
# 1. \n\n(문단 구분)
# 2. \n(줄바꿈)
# 3. " "(공백)
# 4. "" - 최후에는 글자 단위로 chunking
documents = loader.load_and_split(text_splitter=text_splitter)
runtime = time.time() - start
print("문서를 1500글자쯤으로 쪼개면서 읽는 시간 :", runtime)
print("chunk 갯수 :", len(documents))

문서를 1500글자쯤으로 쪼개면서 읽는 시간 : 4.552610158920288
chunk 갯수 : 193


In [8]:
# chunk들의 글자 수
print(max([len(document.page_content) for document in documents]))
print(min([len(document.page_content) for document in documents][:-1]))

1496
873


# 3. 쪼갠 문서를 임베딩 -> 벡터 데이터베이스 저장
- 임베딩 모델 : upstage의 solar-embedding-1-large-passage  solar-embedding-1-large-passage
- 벡터데이터베이스(벡터 store) : chroma

In [9]:
from dotenv import load_dotenv
from langchain_upstage import UpstageEmbeddings
import os
load_dotenv()
embedding = UpstageEmbeddings(
    # api_key=os.getenv('UPSTAGE_API_KEY'),
    model="solar-embedding-1-large-passage")

In [10]:
# embed_query() 한 문자열을 임베딩 벡터로 전환한 숫자 list를 return
len(embedding.embed_query("소득세법은 어쩌구"))

4096

In [12]:
embedding_vectors = embedding.embed_documents(  #여러 문자열을 임베딩 벡터로
    [
        documents[0].page_content,
        documents[1].page_content
    ]
)   # 실행시 과금

In [13]:
print(len(embedding_vectors), len(embedding_vectors[0]),len(embedding_vectors[1]))
print(embedding_vectors[0][:10])

2 4096 4096
[0.007653872482478619, 0.004820186644792557, -0.01941058412194252, 0.003934051375836134, -0.01428203471004963, -0.0034666394349187613, -0.002564274473115802, -0.0016489258268848062, -0.01149054616689682, -0.0008731517009437084]


In [14]:
%%time
from langchain_chroma import Chroma
# 데이터 처음 저장할 때
database = Chroma.from_documents(
    documents=documents,  # chunk
    embedding=embedding,  # 임베딩 객체
    collection_name="tax-collection",  # 생략시 이름 랜덤
    persist_directory="./chroma_upstage"       # 생략시 로컬DB에 저장 안됨. 프로그램 종료시 DB날라감
)   # 저장(한번만실행)
# 이미 저장된 vector DB(store)를 사용할 때
# database = Chroma(
#     embedding_function=embedding, # 질문할때 필요해서
#     collection_name="tax-collection",    
#     persist_directory="./chroma_upstage"
# )

CPU times: total: 4.33 s
Wall time: 38.6 s


In [40]:
results = database._collection.get(include=['embeddings','documents','metadatas'])
print("데이터 수 :", len(results['ids']))
print("문서 임베딩 차원수 :", len(results['embeddings'][0]))
print("1번째 임베딩 샘플 :", results['embeddings'][1])
print("1번째 원본 :", results['documents'][1][:50])
print("1번째 metadata :",results['metadatas'][1])

데이터 수 : 180
문서 임베딩 차원수 : 3072
1번째 임베딩 샘플 : [ 0.01991478 -0.01470464 -0.00057961 ...  0.0058937  -0.03365059
 -0.00657188]
1번째 원본 : . 구성원 간 이익의 분배비율이 정하여져 있지 아니하나 사실상 구성원별로 이익이 분배되는 
1번째 metadata : {'source': './data/소득세법(법률)(제21065호)(20260102).docx'}


# 4. vector DB에 질문과 유사도 검색(답변 생성을 위한 retrieval)

In [15]:
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
retrieved_docs = database.similarity_search(query=query,
                                           k=2)  # 기본 k값은 4

In [44]:
# retrieved_docs

In [22]:
# print("\n\n---\n\n".join([doc.page_content for doc in retrieved_docs]))
retrieved_doc = "\n\n---\n\n".join([doc.page_content for doc in retrieved_docs])

# 5. 유사도 검색으로 가져온 문서를 질문과 같이 LLM에 전달하여 답변 생성-1

In [23]:
from langchain_openai import ChatOpenAI
load_dotenv()
llm = ChatOpenAI(model="gpt-5-mini")

In [24]:
# prompt = f"""[identity]
# - 당신은 최고의 한국 소득세 전문가입니다
# - [context]를 참고해서 사용자의 질문에 답변해 주세요
# [context]는 다음과 같아요
# {retrieved_doc}
# 질문 : {query}"""
prompt = f"""너는 대한민국 세법(특히 소득세법)에 특화된 법령 분석 AI다.
반드시 아래 원칙을 따른다:
1. 제공된 문서(Context)에 근거한 내용만 답변한다.
2. 문서에 명시되지 않은 내용은 추론하거나 일반화하지 않는다.
3. 문서에서 근거를 찾을 수 없는 경우, [제공된 소득세법 문서에는 해당 내용이 명시되어 있지 않습니다]라고 답한다.
[context]는 다음과 같다.
{retrieved_doc}
질문 : {query}"""

In [25]:
ai_message = llm.invoke(prompt)

In [26]:
ai_message.usage_metadata

{'input_tokens': 2063,
 'output_tokens': 1362,
 'total_tokens': 3425,
 'input_token_details': {'audio': 0, 'cache_read': 0},
 'output_token_details': {'audio': 0, 'reasoning': 960}}

In [27]:
print(ai_message.content)

제공된 문서(문맥)만을 근거로 판단하면, 연봉 5천만원인 직장인의 소득세액을 계산할 수 있는 충분한 정보가 문서에 포함되어 있지 않습니다.

근거(문서 내용 일부 인용)
- 제47조(근로소득공제)에는 "근로소득이 있는 거주자에 대해서는 해당 과세기간에 받는 총급여액에서 다음의 금액을 공제한다. 다만, 공제액이 2천만원을 초과하는 경우에는 2천만원을 공제한다."라고 규정되어 있으나, "다음의 금액"을 구체적으로 정한 표·계산식은 제공 문서에 포함되어 있지 않습니다.
- 제공 문서에는 소득세율표(과세표준구간별 세율), 기타 인적공제·표준공제·특별공제 항목의 구체적 금액, 지방소득세 적용률 등 소득세 산출에 필수적인 요소들이 제시되어 있지 않습니다.

따라서 요청하신 "연봉 5천만원인 직장인의 소득세"를 계산하려면 다음과 같은 추가 자료가 필요합니다(참고로 문서에는 이들 자료가 없습니다).
- 근로소득공제의 구체적 계산표(총급여액 구간별 공제액)
- 소득세 과세표준별 세율표 및 세율 적용 방식
- 해당 근로자의 인적공제(본인·부양가족 등), 기타 적용 가능한 공제·세액공제 항목 정보
- 지방소득세 등 부가 세율 여부

[제공된 소득세법 문서에는 해당 내용이 명시되어 있지 않습니다]


# 5. 유사도 검색으로 가져온 문서를 질문과 같이 LLM에 전달하여 답변 생성-2

In [57]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4.1-nano")
promptTemplate = ChatPromptTemplate([
    ("system","당신은 최고의 한국 소득세 전문가입니다"),
    ("human",f"""다음 문맥을 참고하여 질문에 답변하세요.
    답을 모르면 모른다고 말하세요.
    최대 3문장으로 간결하게 답변하세요.
    질문 : {{question}}
    문맥 : {{context}}
    답변 : """)
])
promptTemplate

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 최고의 한국 소득세 전문가입니다'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='다음 문맥을 참고하여 질문에 답변하세요.\n    답을 모르면 모른다고 말하세요.\n    최대 3문장으로 간결하게 답변하세요.\n    질문 : {question}\n    문맥 : {context}\n    답변 : '), additional_kwargs={})])

In [59]:
prompt = promptTemplate.invoke({'context':retrieved_doc, # retrieved_docs보다 추천
                                'question':query
                               })

In [60]:
llm.invoke(prompt)

AIMessage(content='연봉 5천만원인 직장인의 소득세는 과세 표준과 세율에 따라 달라지며, 정확한 금액은 상세한 소득공제 및 세액 공제 등을 고려해야 합니다. 대략적으로 근로소득세율은 6%에서 45% 구간에 속하며, 연간 세액은 약 600만 원 내외가 될 수 있습니다. 정확한 계산을 위해서는 구체적인 공제 내용과 세법에 따른 적용이 필요합니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 111, 'prompt_tokens': 2155, 'total_tokens': 2266, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_f0bc439dc3', 'id': 'chatcmpl-CvasJHPONGTdr2csy2e0Kyd5Prngw', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b9b9f-7951-7892-ab07-87e144663336-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 2155, 'output_tokens': 111, 'total_tokens': 2266, 'input_token_details': {'audio': 0, 'cache_read'

In [62]:
# llm.invoke(promptTemplate.invoke({'context':retrieved_doc,'question':query}))
# 위의 예제를 한번에
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(llm.invoke(promptTemplate.invoke({'context':retrieved_doc,'question':query})))


'연봉 5천만원인 직장인의 소득세는 연간 근로소득공제와 기본세율에 따라 계산되며, 대략 7~8%인 350~400만원 정도입니다. 정확한 금액은 공제 항목과 세율에 따라 달라질 수 있습니다. 상세 계산을 위해서는 구체적인 공제액과 세액 공제 항목이 필요합니다.'

# 6. langchain으로 답변 생성

In [63]:
# 위의 예제를 langchain으로 답변생성
rag_chain = promptTemplate | llm | output_parser
rag_chain.invoke({'context':retrieved_doc,'question':query})

'연봉 5천만원인 직장인의 소득세는 근로소득 공제 후 과세표준에 따라 차등 부과됩니다. 2023년 기준으로 근로소득공제, 기본공제 등을 반영하면 약 400만 원 내외의 세금이 부과될 가능성이 높습니다. 정확한 계산을 위해서는 근로소득 공제액과 세율표를 적용하는 것이 필요합니다.'

## langchain 전달
```  query -> retrieved 전달 -> prompt에 context에 삽입 
```
    smith.langchain.com에서 key생성 후 .env에 LANGCHAIN_API_KEY 추가
    

In [29]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from dotenv import load_dotenv

# 1. LLM과 임베딩 초기화
load_dotenv()
# llm = ChatOpenAI(model="gpt-4.1-mini")
from langchain_upstage import ChatUpstage
llm = ChatUpstage(model="solar-pro2")
                  
embedding = UpstageEmbeddings( model="solar-embedding-1-large-passage")
# 2. vector store load
vectorstore = Chroma(
    embedding_function=embedding,
    collection_name="tax-collection",
    persist_directory="./chroma_upstage"
)
# 3. Retriever 생성
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k":4})
# vectorstore.similarity_search("질문", k=2) - 어떤 질문이 들어올지 몰라서 쓸수 없다.

# 4.프롬프트 템플릿
template = f"""당신은 최고의 한국 소득세 전문가입니다
다음 문맥을 참고하여 질문에 답하세요.
답을 모르면 모른다고 답하세요.
최대 3문장으로 간결하게 답변하세요.
질문 : {{query}}
문맥 : {{context}}
답변 : """
prompt = ChatPromptTemplate.from_template(template)
# 5. 검색된 document를 텍스트로 변환하는 함수
def format_documents(documents):
    return "\n\n---\n\n".join([doc.page_content for doc in documents])

In [30]:
# 6. RAG 체인 구성(LCEL 방식)
from langchain_core.runnables import RunnablePassthrough  # {"query":"~"}  => "~"
rag_chain = (
    {
        "context":retriever | format_documents,
        "query":RunnablePassthrough()  # 질문 그대로 전달
    }
    | prompt  # prompt에 context와 query 변수 주입
    | llm     # llm에 
    | StrOutputParser()
)
# 7. 실행
query = "연봉 5천만원인 직장인의 소득세는 얼마인가요?"
rag_chain.invoke(query)

'연봉 5천만원인 직장인의 소득세는 근로소득공제, 누진세율, 세액공제 등을 종합적으로 적용해야 하며, 제공된 문맥만으로는 정확한 세액을 계산할 수 없습니다. 자녀세액공제(제59조의2)와 근로소득세액공제(제59조) 등 추가 정보가 필요합니다. 정확한 계산을 위해서는 국세청 홈텍스 또는 세무 전문가와 상담이 필요합니다.'